In [2]:
import codecs

with open(r"../arsenal-8/output.txt", "rt") as f:
    last = None
    A = []
    B = []
    for line in f:
        if last != None:
            line = codecs.decode(line.strip().replace(" ", ""), "hex")
            if last == 0:
                A.append(line)
            else:
                B.append(line)
            last = None
        elif line.startswith("ciphertext for 0 is:"):
            last = 0
        elif line.startswith("ciphertext for 1 is:"):
            last = 1
print(len(A), len(B))

63 63


In [4]:
S = bytes([0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
        0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
        0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
        0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
        0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
        0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
        0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
        0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
        0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
        0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
        0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
        0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
        0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
        0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
        0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
        0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16])

invS = [0] * 256
for i in range(256):
    invS[S[i]] = i

rcon = [bytes([x>>24, 0, 0, 0]) for x in [0x01000000, 0x02000000, 0x04000000, 0x08000000, 0x10000000,
        0x20000000, 0x40000000, 0x80000000, 0x1B000000, 0x36000000]]

M = [[0x02, 0x03, 0x01, 0x01],
                       [0x01, 0x02, 0x03, 0x01],
                       [0x01, 0x01, 0x02, 0x03],
                       [0x03, 0x01, 0x01, 0x02]]

invM = [[0x0E, 0x0B, 0x0D, 0x09], [0x09, 0x0E, 0x0B, 0x0D], [0x0D, 0x09, 0x0E, 0x0B], [0x0B, 0x0D, 0x09, 0x0E]]

def keyexpand(key: bytes) -> list:
    assert len(key) == 16
    k = [bytearray([0, 0, 0, 0]) for i in range(4*11)]
    for i in range(16):
        k[i//4][i%4] = key[i]

    for i in range(10):
        k[i*4+4] = bytes([k[i*4+0][j] ^ S[k[i*4+3][(j+1)%4]] ^ rcon[i][j] for j in range(4)])
        k[i*4+5] = bytes([k[i*4+1][j] ^ k[i*4+4][j] for j in range(4)])
        k[i*4+6] = bytes([k[i*4+2][j] ^ k[i*4+5][j] for j in range(4)])
        k[i*4+7] = bytes([k[i*4+3][j] ^ k[i*4+6][j] for j in range(4)])

    return k

def invkeyexpand(k: list, baseround: int) -> bytes:
    assert len(k) >= baseround*4 + 4

    for i in range(baseround-1, -1, -1):
        k[i*4+1] = bytes([k[i*4+5][j] ^ k[i*4+4][j] for j in range(4)])
        k[i*4+2] = bytes([k[i*4+6][j] ^ k[i*4+5][j] for j in range(4)])
        k[i*4+3] = bytes([k[i*4+7][j] ^ k[i*4+6][j] for j in range(4)])
        k[i*4+0] = bytes([k[i*4+4][j] ^ S[k[i*4+3][(j+1)%4]] ^ rcon[i][j] for j in range(4)])
    
    return [k[i//4][i%4] for i in range(16)]

def symbolized_keyexpand(rounds:int=0, baseround:int=0) -> list:
    k = [[None, None, None, None] for i in range(4*(rounds+1))]
    for i in range(16):
        k[baseround*4+i//4][i%4] = f"K[{i}]"
    for i in range(baseround, rounds):
        k[i*4+4] = [k[i*4+0][j] + "^" + "S[" + k[i*4+3][(j+1)%4] + "]" + ("^" + hex(rcon[i][j]) if rcon[i][j] else "") for j in range(4)]
        k[i*4+5] = [k[i*4+1][j] + "^" + k[i*4+4][j] for j in range(4)]
        k[i*4+6] = [k[i*4+2][j] + "^" + k[i*4+5][j] for j in range(4)]
        k[i*4+7] = [k[i*4+3][j] + "^" + k[i*4+6][j] for j in range(4)]
    return k

def addroundkey(A, round:int):
    size = len(A)
    return [[A[i][j] ^ k[j+round*size][i] for j in range(size)] for i in range(size)]

def shiftrows(A):
    size = len(A)
    return [[A[i][(j+i)%size] for j in range(size)] for i in range(size)]

def subbytes(A):
    size = len(A)
    return [[S[A[i][j]] for j in range(size)] for i in range(size)]

def GMul(u: int, v: int):
    assert u & 0xFF == u
    assert v & 0xFF == v
    p = 0
    for i in range(8):
        if u & 0x1:
            p = p ^ v
        flag = v & 0x80
        v = v << 1
        if flag:
            v = v ^ 0x1B
        u = u >> 1
    return p & 0xFF

def GAdd(u: int, v: int)->int:
    assert u & 0xFF == u
    assert v & 0xFF == v
    if (u+v) >= 0x100:
        return (u+v) ^ 0x11B
    else:
        return u+v

def xorall(t:list) -> int:
    result = 0
    for x in t:
        result = result ^ x
    return result

def mix(i:int, column:bytes, size:int=4) -> int:
    return GMul(M[i][0], column[0]) ^ GMul(M[i][1], column[1]) ^ GMul(M[i][2], column[2]) ^ GMul(M[i][3], column[3])

def invmix(i:int, column:bytes, size:int=4) -> int:
    return GMul(invM[i][0], column[0]) ^ GMul(invM[i][1], column[1]) ^ GMul(invM[i][2], column[2]) ^ GMul(invM[i][3], column[3])

def MIX0(a:int, b:int, c:int, d:int) -> int:
    return mix(0, bytes([a,b,c,d]))

def MIX1(a:int, b:int, c:int, d:int) -> int:
    return mix(1, bytes([a,b,c,d]))

def MIX2(a:int, b:int, c:int, d:int) -> int:
    return mix(2, bytes([a,b,c,d]))

def MIX3(a:int, b:int, c:int, d:int) -> int:
    return mix(3, bytes([a,b,c,d]))

def mixcolumns(A):
    size = len(A)
    return [[mix(i, bytes([A[k][j] for k in range(size)]), size) for j in range(len(A[i]))] for i in range(size)]

def invmixcolumns(A):
    size = len(A)
    return [[invmix(i, bytes([A[k][j] for k in range(size)]), size) for j in range(len(A[i]))] for i in range(size)]

def symbolized_xor(x:str, y:str):
    if x != "0":
        return f"{x}^{y}"
    else:
        return y

def symbolized_addroundkey(A, round:int):
    size = len(A)
    return [[symbolized_xor(A[i][j], "k{r}{i}".format(i=i, r=j+round*size)) for j in range(size)] for i in range(size)]

def symbolized_shiftrows(A):
    size = len(A)
    return [[A[i][(j+i)%size] for j in range(size)] for i in range(size)]

def symbolized_subbytes(A):
    size = len(A)
    return [[f"S[{A[i][j]}]" for j in range(size)] for i in range(size)]

def symbolized_mixcolumns(A):
    size = len(A)
    return [[f"MIX{i}(" + ",".join(A[k][j] for k in range(size)) + ")" for j in range(size)] for i in range(size)]

def symbolized_aes(rounds:int=3, size:int=4, iposition:int=1):
    A = [["0" for j in range(size)] for i in range(size)]
    A[iposition%4][iposition//4] = "i"

    # round 0
    A = symbolized_addroundkey(A, 0)

    for round in range(1, rounds+1):
        A = symbolized_shiftrows(A)
        A = symbolized_subbytes(A)
        A = symbolized_mixcolumns(A)
        A = symbolized_addroundkey(A, round)

    A = symbolized_shiftrows(A)
    A = symbolized_subbytes(A)
    A = symbolized_addroundkey(A, rounds+1)

    return A


def aes(A, rounds:int=3, size:int=4):
    if type(A) is str:
        assert len(A) == 16
        A = A.encode()
    if type(A) is list:
        A = bytes(A)
    if type(A) is bytes:
        assert len(A) == 16
        tmp = [bytearray([0, 0, 0, 0]) for i in range(4)]
        for i in range(16):
            tmp[i%4][i//4] = A[i]
        A = tmp

    # round 0
    A = addroundkey(A, 0)

    for round in range(1, rounds+1):
        A = shiftrows(A)
        A = subbytes(A)
        A = mixcolumns(A)
        A = addroundkey(A, round)

    A = shiftrows(A)
    A = subbytes(A)
    A = addroundkey(A, rounds+1)

    return [A[i%4][i//4] for i in range(16)]

In [5]:
cachedoutput = {}

In [6]:
def solverfor3rounds(ciphertextarray:list[bytes], iposition:int) -> bytes:
    import subprocess

    k = [[0]*4 for i in range(20)]

    perbyteexpressions = symbolized_aes(3, 4, iposition)
    bytegroups = [[0, 7, 10, 13], [1, 4, 11, 14], [2, 5, 8, 15], [3, 6, 9, 12]]

    for bytegroup in bytegroups:
        assert len(set([perbyteexpressions[i%4][i//4][10:80] for i in bytegroup])) == 1

        valuesinfo = {}

        for i in bytegroup:
            expression = perbyteexpressions[i%4][i//4]
            # S[MIX3(S[MIX0(S[MIX0(S[k20],S[k31],S[k02],S[k13])^k60],S[MIX1(S[k30],S[i^k01],S[k12],S[k23])^k71],S[MIX2(S[k00],S[k11],S[k22],S[k33])^k42],S[MIX3(S[k10],S[k21],S[k32],S[k03])^k53])^k100],S[MIX1(S[MIX0(S[k30],S[i^k01],S[k12],S[k23])^k70],S[MIX1(S[k00],S[k11],S[k22],S[k33])^k41],S[MIX2(S[k10],S[k21],S[k32],S[k03])^k52],S[MIX3(S[k20],S[k31],S[k02],S[k13])^k63])^k111],S[MIX2(S[MIX0(S[k00],S[k11],S[k22],S[k33])^k40],S[MIX1(S[k10],S[k21],S[k32],S[k03])^k51],S[MIX2(S[k20],S[k31],S[k02],S[k13])^k62],S[MIX3(S[k30],S[i^k01],S[k12],S[k23])^k73])^k82],S[MIX3(S[MIX0(S[k10],S[k21],S[k32],S[k03])^k50],S[MIX1(S[k20],S[k31],S[k02],S[k13])^k61],S[MIX2(S[k30],S[i^k01],S[k12],S[k23])^k72],S[MIX3(S[k00],S[k11],S[k22],S[k33])^k43])^k93])^k143]^k193
        
            kindices = expression.rsplit("^",2)[-2:] # "k143]", "k193"
            kindices = [(int(kindex[:-1]), int(kindex[-1])) for kindex in [kindex.strip("k]") for kindex in kindices]]

            outermixnum = int(expression[5])
            valuesinfo[outermixnum] = {"kindices": kindices, "values": ",".join(str(ciphertextarray[j][i]) for j in range(256))}

        expression = expression.split("(",1)[1].rsplit(")",1)[0] # any expression in this bytegroup will do
        # S[MIX0(S[MIX0(S[k20],S[k31],S[k02],S[k13])^k60],S[MIX1(S[k30],S[i^k01],S[k12],S[k23])^k71],S[MIX2(S[k00],S[k11],S[k22],S[k33])^k42],S[MIX3(S[k10],S[k21],S[k32],S[k03])^k53])^k100],S[MIX1(S[MIX0(S[k30],S[i^k01],S[k12],S[k23])^k70],S[MIX1(S[k00],S[k11],S[k22],S[k33])^k41],S[MIX2(S[k10],S[k21],S[k32],S[k03])^k52],S[MIX3(S[k20],S[k31],S[k02],S[k13])^k63])^k111],S[MIX2(S[MIX0(S[k00],S[k11],S[k22],S[k33])^k40],S[MIX1(S[k10],S[k21],S[k32],S[k03])^k51],S[MIX2(S[k20],S[k31],S[k02],S[k13])^k62],S[MIX3(S[k30],S[i^k01],S[k12],S[k23])^k73])^k82],S[MIX3(S[MIX0(S[k10],S[k21],S[k32],S[k03])^k50],S[MIX1(S[k20],S[k31],S[k02],S[k13])^k61],S[MIX2(S[k30],S[i^k01],S[k12],S[k23])^k72],S[MIX3(S[k00],S[k11],S[k22],S[k33])^k43])^k93]

        abcd = [None] * 4
        for j in range(4):
            abcd[j] = expression.split(f"S[MIX{j+1}(S[MIX0(")[0]
            expression = expression[len(abcd[j]):]

        print("A", abcd[0])
        print("B", abcd[1])
        print("C", abcd[2])

        mulinfo = {}

        for j in range(4):
            expression = abcd[j]
            ipos = expression.find("i^")
            innermixpos = expression[:ipos].rfind("MIX")

            mul2 = M[int(expression[innermixpos+3])][expression[innermixpos+3:ipos].count(",")]
            #print(i, mul2)

            mul1 = M[int(expression[5])][expression[5:ipos].count("^k")]
            print(bytegroup, j, mul1, mul2)

            mulinfo[j] = (mul1, mul2)

        print("kindices", [valuesinfo[i]["kindices"] for i in range(4)])

        args = [f"{mulinfo[i][0]},{mulinfo[i][1]}" for i in range(4)] + [valuesinfo[i]["values"] for i in range(4)]
        if "|".join(args) not in cachedoutput:
            cachedoutput["|".join(args)] = subprocess.check_output(["equationsolver/bin/Release/equationsolver.exe"] + args)
        output = cachedoutput["|".join(args)]
        print(output)

        # bingo k1=%d ok1=%d ok2=%d ok3=%d ok4=%d
        assert b"bingo" in output
        resultvars = {}
        for line in output.decode().split("\n"):
            if line.startswith("bingo"):
                resultvars = {item.split("=",1)[0]: int(item.split("=",1)[1]) for item in line.rstrip().split(" ")[1:]}
                break
        
        for i in range(4):
            kindex = valuesinfo[i]["kindices"][1]
            kvalue = resultvars[f"ok{i+1}"]
            k[kindex[0]][kindex[1]] = kvalue

    for i in range(16, 20):
        print(f"k{i} = {k[i]}")

    return invkeyexpand(k, 4)

In [8]:
key = solverfor3rounds([bytes(A[i]) for i in range(len(A))] + [bytes([0]*16) for i in range(len(A), 256)], 8)
print(key)

A S[MIX0(S[MIX0(S[k00],S[k11],S[k22],S[k33])^k40],S[MIX1(S[k10],S[k21],S[k32],S[k03])^k51],S[MIX2(S[i^k20],S[k31],S[k02],S[k13])^k62],S[MIX3(S[k30],S[k01],S[k12],S[k23])^k73])^k80],
B S[MIX1(S[MIX0(S[k10],S[k21],S[k32],S[k03])^k50],S[MIX1(S[i^k20],S[k31],S[k02],S[k13])^k61],S[MIX2(S[k30],S[k01],S[k12],S[k23])^k72],S[MIX3(S[k00],S[k11],S[k22],S[k33])^k43])^k91],
C S[MIX2(S[MIX0(S[i^k20],S[k31],S[k02],S[k13])^k60],S[MIX1(S[k30],S[k01],S[k12],S[k23])^k71],S[MIX2(S[k00],S[k11],S[k22],S[k33])^k42],S[MIX3(S[k10],S[k21],S[k32],S[k03])^k53])^k102],
[0, 7, 10, 13] 0 1 1
[0, 7, 10, 13] 1 2 1
[0, 7, 10, 13] 2 1 2
[0, 7, 10, 13] 3 2 3
kindices [[(12, 0), (16, 0)], [(12, 1), (19, 1)], [(12, 2), (18, 2)], [(12, 3), (17, 3)]]
b'196 19 68 179\r\n154 123 252 205\r\nmaxdup in lookup table: 0\r\nmaxdup in lookup table: 0\r\nmaxdup in lookup table: 0\r\nmaxdup in lookup table: 0\r\nlookup tables built in 2.967 s\r\none: 1.404 s\r\none: 1.406 s\r\none: 1.454 s\r\none: 1.460 s\r\none: 1.363 s\r\none: 1.413 

In [9]:
import codecs
encryptedflag = codecs.decode("a8 8e 94 8e 60 73 29 70 05 28 5e 9d d6 b1 5e 9f 98 10 2c 8f ca 68 64 74 d1 63 38 ff 73 04 b1 a5 ".replace(" ", ""), "hex")
key = [69, 65, 57, 150, 81, 48, 39, 93, 235, 157, 218, 184, 155, 0, 51, 215]
k = keyexpand(key)


In [10]:
def invshiftrows(A):
    size = len(A)
    return [[A[i][(4-i+j)%size] for j in range(size)] for i in range(size)]

def invsubbytes(A):
    size = len(A)
    return [[invS[A[i][j]] for j in range(size)] for i in range(size)]

def aesdecrypt(A, rounds:int=3, size:int=4):
    if type(A) is str:
        assert len(A) == 16
        A = A.encode()
    if type(A) is list:
        A = bytes(A)
    if type(A) is bytes:
        assert len(A) == 16
        tmp = [bytearray([0, 0, 0, 0]) for i in range(4)]
        for i in range(16):
            tmp[i%4][i//4] = A[i]
        A = tmp

    A = addroundkey(A, rounds+1)

    for round in range(rounds, 0, -1):
        A = invshiftrows(A)
        A = invsubbytes(A)
        A = addroundkey(A, round)
        A = invmixcolumns(A)

    A = invsubbytes(A)
    A = invshiftrows(A)
    A = addroundkey(A, 0)

    return [A[i%4][i//4] for i in range(16)]

[bytes(aesdecrypt(encryptedflag[i:i+16])) for i in range(0, len(encryptedflag), 16)]

[b'0CTF{~~1MP0SS1BL', b'E_1S_N0TH1NG!~~}']